In [1]:
import os
from PyPDF2 import PdfReader
from dotted_dict import DottedDict
import numpy as np
from langchain_community.vectorstores import Chroma
from langchain_openai import AzureOpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
from langchain_core.documents import Document
recursive_text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)


c:\LLm_Final_Project\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
def read_pdf(file_path):
    content = ""
    with open(file_path, 'rb') as file:
        reader = PdfReader(file)
        for page in range(len(reader.pages)):
            content += reader.pages[page].extract_text()
    return content

In [3]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
def sem_process_pdfs(pdf_directory):
    documents = []

    for filename in os.listdir(pdf_directory):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(pdf_directory, filename)
            content = read_pdf(pdf_path)

            # Split text into chunks
            split_chunks = recursive_text_splitter.split_text(content)
            text_splitter = SemanticChunker(embeddings)
            split_chunks = text_splitter.create_documents([content])

            # Create Document objects with metadata
            for chunk in split_chunks:
                chunk.metadata['pdf_name'] = filename
                print(type(chunk))
                documents.append(chunk)
    return documents

c:\LLm_Final_Project\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
import pandas as pd
data = []
for doc in documents:
    row = {
        "content": doc.page_content,
        "pdf_name": doc.metadata["pdf_name"]
    }
    data.append(row)

# Create a DataFrame and save to CSV
df = pd.DataFrame(data)
csv_path = "processed_documents.csv"
df.to_csv(csv_path, index=False)